In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import make_scorer
import spacy
import re
import string
from gensim.models import KeyedVectors
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

In [2]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import load_model
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import GRU,Conv1D,MaxPooling1D,GlobalMaxPooling1D, Flatten,Embedding,BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [3]:
import tensorflow as tf

In [4]:
!python -m nltk.downloader all

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nl

In [5]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('drive/MyDrive/cleaned_data_final3.csv')
df_new2 = df[['area_id', 'prof_name', 'name',
            'mean_salary', 'description','schedule', 'experience_rus','employment','quick_responses_allowed','premium',
            'allow_messages','accept_temporary', 'first_language', 'key_skills', 'mean_salary_all', 'city']]

In [8]:
df_new2 = df_new2.drop_duplicates(subset=['description'])

In [9]:
names_to_remove = ['Технический писатель', 'Продуктовый аналитик' , 'Директор по информационным технологиям (CIO)','Дата-сайентист',
                   'Руководитель отдела аналитики', 'Методолог', 'Гейм-дизайнер', 'Арт-директор, креативный директор']
df_new2 = df_new2[~df_new2['prof_name'].isin(names_to_remove)]

In [10]:
class PreprocessData:
    def __init__ (self, data, y, stem_method=Mystem()):
        """Clean data and preprocess text"""
        self.data = data
        self.y = y
        self.mystem = stem_method
        # self.show_stats = show_stats

    def fill_nans(self):
        self.data[y] = self.data[y].fillna(0)
        for col in self.data.select_dtypes(['object']).columns:
            self.data[col] = self.data[col].fillna('')

    def drop_duplicates(self):
        self.data = self.data.drop_duplicates(subset=['description'])

    def clean_and_lemmatize(self, text):
        # clean punctuation and stop words
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)
        stop_words = set(stopwords.words('russian'))
        words = [w for w in words if w not in stop_words]
        # clean_text = ' '.join(words)

        # lemmatize
        # words = text.split()
        lemmas = [self.mystem.lemmatize(word)[0] for word in words]
        lemmatized_text = ' '.join(lemmas)
        return lemmatized_text

    def get_cleaned_data(self):
        self.drop_duplicates()
        self.fill_nans()
        self.data['descr_clean'] = self.data['description'].progress_apply(self.clean_and_lemmatize)
        return self.data.reset_index(drop=True)

    def get_stats(self, cat_features):
        print(self.data.info(), '\n')
        print("Numerical features' analysis")
        plt.figure()
        self.data[self.y].hist(bins=30)
        plt.title(f'{self.y} distribution')
        plt.show;
        display(self.data.describe().T)

        print('\n', "Categorical features' analysis")
        display(self.data.describe(include=object).T)
        for col in cat_features:
            plt.figure()
            self.data[col].value_counts().head(10).plot(kind='barh', color='pink')
            plt.title(f'{col} frequency')
            plt.show();

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [17]:
y = 'mean_salary_all'
cat_features = ['prof_name', 'city', 'schedule', 'employment', 'experience_rus']
text_features = ['key_skills', 'descr_clean', 'name']

In [18]:
data_processed = PreprocessData(df_new2, y)
df_new2 = data_processed.get_cleaned_data()

  0%|          | 0/14335 [00:00<?, ?it/s]

We have several text columns: vacancy name, key skills and description. Let's try to use Bi-GRU-CNN on this data.

In [11]:
df_new2['all_text'] = df_new2['name'].str.lower()  + '. '+ df_new2['key_skills'].fillna('').str.lower() +  '. ' + df_new2['description'].str.lower()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_new2['all_text'].values, df_new2.mean_salary_all.values.astype(int), test_size=0.2, random_state=10)

In [13]:
tokenizer = Tokenizer()
num_rows = df_new2['all_text'].shape[0]
tokenizer.fit_on_texts(df_new2['all_text'].values)
row_max_length = max([len(x.split()) for x in df_new2['all_text'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [14]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='linear'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=128

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1145, 256)         21398272  
                                                                 
 bidirectional (Bidirection  (None, 1145, 256)         296448    
 al)                                                             
                                                                 
 conv1d (Conv1D)             (None, 1145, 64)          49216     
                                                                 
 max_pooling1d (MaxPooling1  (None, 572, 64)           0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 36608)             0         
                                                                 
 dropout (Dropout)           (None, 36608)            

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad,y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/15
90/90 [==============================] - 30s 293ms/step - loss: 51826.5234 - mae: 51826.5234
Epoch 2/15
90/90 [==============================] - 26s 285ms/step - loss: 42157.7109 - mae: 42157.7109
Epoch 3/15
90/90 [==============================] - 27s 298ms/step - loss: 36578.8867 - mae: 36578.8867
Epoch 4/15
90/90 [==============================] - 25s 267ms/step - loss: 30351.8320 - mae: 30351.8320
Epoch 5/15
90/90 [==============================] - 27s 298ms/step - loss: 26208.0332 - mae: 26208.0332
Epoch 6/15
90/90 [==============================] - 22s 243ms/step - loss: 23401.3789 - mae: 23401.3789
Epoch 7/15
90/90 [==============================] - 22s 247ms/step - loss: 20832.3613 - mae: 20832.3613
Epoch 8/15
90/90 [==============================] - 21s 231ms/step - loss: 18751.4414 - mae: 18751.4414
Epoch 9/15
90/90 [==============================] - 21s 236ms/step - loss: 17945.3496 - mae: 17945.3496
Epoch 10/15
90/90 [==============================] - 21s 236ms/s

In [ ]:
scores = model.evaluate(X_test_pad, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 4s 35ms/step - loss: 30529.5254 - mae: 30529.5254
Test MAE: 30529.525390625
90/90 [==============================] - 4s 32ms/step
R2 score: 52.76


Let's see model results without description. Only with name, key skills.

In [ ]:
df_new2['all_text_wo_descr'] = df_new2['name'].str.lower()  + '. '+ df_new2['key_skills'].fillna('').str.lower()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_new2['all_text_wo_descr'].values, df_new2.mean_salary_all.values.astype(int), test_size=0.2, random_state=10)

In [ ]:
tokenizer = Tokenizer()
num_rows = df_new2['all_text_wo_descr'].shape[0]
tokenizer.fit_on_texts(df_new2['all_text_wo_descr'].values)
row_max_length = max([len(x.split()) for x in df_new2['all_text_wo_descr'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_pad1 = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad1 = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='linear'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=128

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad1, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
90/90 [==============================] - 18s 125ms/step - loss: 56542.5430 - mae: 56542.5430
Epoch 2/15
90/90 [==============================] - 6s 64ms/step - loss: 44646.9922 - mae: 44646.9922
Epoch 3/15
90/90 [==============================] - 5s 56ms/step - loss: 38527.9375 - mae: 38527.9375
Epoch 4/15
90/90 [==============================] - 4s 39ms/step - loss: 34515.0312 - mae: 34515.0312
Epoch 5/15
90/90 [==============================] - 8s 92ms/step - loss: 32175.8516 - mae: 32175.8516
Epoch 6/15
90/90 [==============================] - 2s 27ms/step - loss: 30379.9844 - mae: 30379.9844
Epoch 7/15
90/90 [==============================] - 3s 29ms/step - loss: 29340.4707 - mae: 29340.4707
Epoch 8/15
90/90 [==============================] - 2s 24ms/step - loss: 28325.7949 - mae: 28325.7949
Epoch 9/15
90/90 [==============================] - 2s 22ms/step - loss: 27824.6426 - mae: 27824.6426
Epoch 10/15
90/90 [==============================] - 2s 25ms/step - loss: 27179.

In [ ]:
scores = model.evaluate(X_test_pad1, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad1).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 2s 7ms/step - loss: 34189.5742 - mae: 34189.5742
Test MAE: 34189.57421875
90/90 [==============================] - 1s 4ms/step
R2 score: 41.74


Let's see other variables like text except the description

In [ ]:
df_new2['total'] = df_new2['city'] + '. ' + df_new2['schedule'].str.lower() +'. '+ df_new2['name'] + '. ' + df_new2['employment'].str.lower() +'. ' +  df_new2['prof_name'].str.lower()  + '. ' + df_new2['experience_rus'].str.lower() +'. '+ df_new2['key_skills'].fillna('')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_new2['total'].values, df_new2.mean_salary_all.values.astype(int), test_size=0.2, random_state=10)

In [ ]:
tokenizer = Tokenizer()
num_rows = df_new2['total'].shape[0]
tokenizer.fit_on_texts(df_new2['total'].values)
row_max_length = max([len(x.split()) for x in df_new2['total'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_pad2 = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad2 = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='linear'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=128

In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='linear'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=128

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad2, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
90/90 [==============================] - 22s 167ms/step - loss: 56213.3086 - mae: 56213.3086
Epoch 2/15
90/90 [==============================] - 5s 58ms/step - loss: 38500.2188 - mae: 38500.2188
Epoch 3/15
90/90 [==============================] - 4s 41ms/step - loss: 30107.4062 - mae: 30107.4062
Epoch 4/15
90/90 [==============================] - 4s 47ms/step - loss: 26740.2070 - mae: 26740.2070
Epoch 5/15
90/90 [==============================] - 4s 44ms/step - loss: 24624.1699 - mae: 24624.1699
Epoch 6/15
90/90 [==============================] - 3s 33ms/step - loss: 23805.9883 - mae: 23805.9883
Epoch 7/15
90/90 [==============================] - 3s 30ms/step - loss: 22898.1426 - mae: 22898.1426
Epoch 8/15
90/90 [==============================] - 3s 28ms/step - loss: 22107.0078 - mae: 22107.0078
Epoch 9/15
90/90 [==============================] - 2s 28ms/step - loss: 21499.4824 - mae: 21499.4824
Epoch 10/15
90/90 [==============================] - 2s 26ms/step - loss: 20915.

In [ ]:
scores = model.evaluate(X_test_pad2, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad2).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 2s 7ms/step - loss: 26546.1992 - mae: 26546.1992
Test MAE: 26546.19921875
90/90 [==============================] - 1s 5ms/step
R2 score: 63.74


Let's see add description

In [19]:
df_new2['total'] = df_new2['city'] + '. ' + df_new2['schedule'].str.lower() +'. '+ df_new2['name'] + '. ' + df_new2['employment'].str.lower() +'. ' +  df_new2['prof_name'].str.lower()  + '. ' + df_new2['experience_rus'].str.lower() +'. '+ df_new2['key_skills'].fillna('') + '. '+ df_new2['descr_clean'].str.lower()

In [20]:
tokenizer = Tokenizer()
num_rows = df_new2['total'].shape[0]
tokenizer.fit_on_texts(df_new2['total'].values)
row_max_length = max([len(x.split()) for x in df_new2['total'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [21]:
X_train_pad3 = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad3 = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [22]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=128

In [23]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad3, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
90/90 [==============================] - 28s 215ms/step - loss: 51476.5000 - mae: 51476.5000
Epoch 2/15
90/90 [==============================] - 18s 198ms/step - loss: 37311.6602 - mae: 37311.6602
Epoch 3/15
90/90 [==============================] - 18s 199ms/step - loss: 30579.0586 - mae: 30579.0586
Epoch 4/15
90/90 [==============================] - 17s 186ms/step - loss: 27110.4648 - mae: 27110.4648
Epoch 5/15
90/90 [==============================] - 16s 174ms/step - loss: 23790.3711 - mae: 23790.3711
Epoch 6/15
90/90 [==============================] - 16s 172ms/step - loss: 21446.5000 - mae: 21446.5000
Epoch 7/15
90/90 [==============================] - 16s 179ms/step - loss: 19769.2148 - mae: 19769.2148
Epoch 8/15
90/90 [==============================] - 16s 181ms/step - loss: 18372.6387 - mae: 18372.6387
Epoch 9/15
90/90 [==============================] - 15s 165ms/step - loss: 17119.9883 - mae: 17119.9883
Epoch 10/15
90/90 [==============================] - 14s 160ms/s

In [25]:
scores = model.evaluate(X_test_pad3, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad3).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2, 2))

90/90 [==============================] - 3s 33ms/step - loss: 28740.3828 - mae: 28740.3828
Test MAE: 28740.3828125
90/90 [==============================] - 2s 26ms/step
R2 score: 0.57


In [26]:
def clean_review_text(text):

    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word not in stop_words]
    clean_text = ' '.join(words)

    return clean_text


mystem = Mystem()

def lemmatize_text(text):
    words = text.split()
    lemmas = [mystem.lemmatize(word)[0] for word in words]
    lemmatized_text = ' '.join(lemmas)

    return lemmatized_text
df_new2['text_clean'] = df_new2['description'].apply(clean_review_text)

df_new2['lemmatized_text'] = df_new2['text_clean'].apply(lemmatize_text)

In [27]:
df_new2['total'] = df_new2['city'] + '. ' + df_new2['schedule'].str.lower() +'. '+ df_new2['name'] + '. ' + df_new2['employment'].str.lower() +'. ' +  df_new2['prof_name'].str.lower()  + '. ' + df_new2['experience_rus'].str.lower() +'. '+ df_new2['key_skills'].fillna('') + '. '+ df_new2['lemmatized_text'].str.lower()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df_new2['total'].values, df_new2.mean_salary_all.values.astype(int), test_size=0.2, random_state=10)

In [29]:
tokenizer = Tokenizer()
num_rows = df_new2['total'].shape[0]
tokenizer.fit_on_texts(df_new2['total'].values)
row_max_length = max([len(x.split()) for x in df_new2['total'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [30]:
X_train_pad5 = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad5 = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [31]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=64

In [32]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad5, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
180/180 [==============================] - 42s 205ms/step - loss: 47724.7930 - mae: 47724.7930
Epoch 2/15
180/180 [==============================] - 26s 147ms/step - loss: 29606.8457 - mae: 29606.8457
Epoch 3/15
180/180 [==============================] - 26s 144ms/step - loss: 24901.1602 - mae: 24901.1602
Epoch 4/15
180/180 [==============================] - 23s 126ms/step - loss: 21298.4805 - mae: 21298.4805
Epoch 5/15
180/180 [==============================] - 22s 121ms/step - loss: 19280.3984 - mae: 19280.3984
Epoch 6/15
180/180 [==============================] - 21s 115ms/step - loss: 18268.6855 - mae: 18268.6855
Epoch 7/15
180/180 [==============================] - 21s 115ms/step - loss: 16435.6641 - mae: 16435.6641
Epoch 8/15
180/180 [==============================] - 19s 104ms/step - loss: 14789.7275 - mae: 14789.7275
Epoch 9/15
180/180 [==============================] - 19s 105ms/step - loss: 13657.8877 - mae: 13657.8877
Epoch 10/15
180/180 [=========================

In [33]:
scores = model.evaluate(X_test_pad5, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad5).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 4s 27ms/step - loss: 25705.9121 - mae: 25705.9121
Test MAE: 25705.912109375
90/90 [==============================] - 3s 24ms/step
R2 score: 64.25


In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.0101
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=20
batch_size=64

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad5, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/20
180/180 [==============================] - 35s 169ms/step - loss: 49301.7891 - mae: 49301.7891
Epoch 2/20
180/180 [==============================] - 28s 153ms/step - loss: 31638.9082 - mae: 31638.9082
Epoch 3/20
180/180 [==============================] - 26s 145ms/step - loss: 24556.5977 - mae: 24556.5977
Epoch 4/20
180/180 [==============================] - 22s 123ms/step - loss: 21476.2520 - mae: 21476.2520
Epoch 5/20
180/180 [==============================] - 23s 125ms/step - loss: 19701.7207 - mae: 19701.7207
Epoch 6/20
180/180 [==============================] - 22s 124ms/step - loss: 17887.7520 - mae: 17887.7520
Epoch 7/20
180/180 [==============================] - 20s 112ms/step - loss: 16485.5605 - mae: 16485.5605
Epoch 8/20
180/180 [==============================] - 19s 108ms/step - loss: 15728.4492 - mae: 15728.4492
Epoch 9/20
180/180 [==============================] - 20s 112ms/step - loss: 14840.9971 - mae: 14840.9971
Epoch 10/20
180/180 [=========================

In [ ]:
scores = model.evaluate(X_test_pad5, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad5).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2, 3))

90/90 [==============================] - 3s 28ms/step - loss: 25967.3145 - mae: 25967.3145
Test MAE: 25967.314453125
90/90 [==============================] - 3s 24ms/step
R2 score: 0.656


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_new2['lemmatized_text'].values, df_new2.mean_salary_all.values.astype(int), test_size=0.2, random_state=10)

In [ ]:
tokenizer = Tokenizer()
num_rows = df_new2['lemmatized_text'].shape[0]
tokenizer.fit_on_texts(df_new2['lemmatized_text'].values)
row_max_length = max([len(x.split()) for x in df_new2['lemmatized_text'].values])
vocabulary_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_pad4 = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad4 = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')

In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=64

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad4, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
180/180 [==============================] - 39s 186ms/step - loss: 49603.5391 - mae: 49603.5391
Epoch 2/15
180/180 [==============================] - 29s 161ms/step - loss: 33941.1836 - mae: 33941.1836
Epoch 3/15
180/180 [==============================] - 30s 169ms/step - loss: 25378.2383 - mae: 25378.2383
Epoch 4/15
180/180 [==============================] - 23s 129ms/step - loss: 21924.9746 - mae: 21924.9746
Epoch 5/15
180/180 [==============================] - 23s 128ms/step - loss: 19553.7344 - mae: 19553.7344
Epoch 6/15
180/180 [==============================] - 21s 118ms/step - loss: 17622.8359 - mae: 17622.8359
Epoch 7/15
180/180 [==============================] - 21s 117ms/step - loss: 15982.1455 - mae: 15982.1455
Epoch 8/15
180/180 [==============================] - 20s 114ms/step - loss: 15181.3984 - mae: 15181.3984
Epoch 9/15
180/180 [==============================] - 20s 109ms/step - loss: 14670.9004 - mae: 14670.9004
Epoch 10/15
180/180 [=========================

In [ ]:
scores = model.evaluate(X_test_pad4, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad4).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 3s 27ms/step - loss: 25851.3457 - mae: 25851.3457
Test MAE: 25851.345703125
90/90 [==============================] - 3s 28ms/step
R2 score: 65.7


In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model1=Sequential()
model1.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model1.add(Bidirectional(GRU(units=128, return_sequences=True)))
model1.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Flatten())
model1.add(Dropout(0.2))
model1.add(Dense(units = 1, activation='relu'))
model1.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=12
batch_size=128

In [ ]:
with tf.device('/gpu:0'):
  model1.fit(X_train_pad4, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/12
90/90 [==============================] - 24s 224ms/step - loss: 51752.2500 - mae: 51752.2500
Epoch 2/12
90/90 [==============================] - 19s 208ms/step - loss: 44561.9375 - mae: 44561.9375
Epoch 3/12
90/90 [==============================] - 19s 213ms/step - loss: 39753.5781 - mae: 39753.5781
Epoch 4/12
90/90 [==============================] - 18s 197ms/step - loss: 31099.4102 - mae: 31099.4102
Epoch 5/12
90/90 [==============================] - 17s 194ms/step - loss: 25540.5586 - mae: 25540.5586
Epoch 6/12
90/90 [==============================] - 17s 190ms/step - loss: 21755.2500 - mae: 21755.2500
Epoch 7/12
90/90 [==============================] - 17s 187ms/step - loss: 19667.3438 - mae: 19667.3438
Epoch 8/12
90/90 [==============================] - 16s 179ms/step - loss: 18282.4355 - mae: 18282.4355
Epoch 9/12
90/90 [==============================] - 18s 201ms/step - loss: 16655.7246 - mae: 16655.7246
Epoch 10/12
90/90 [==============================] - 16s 178ms/s

In [ ]:
scores = model1.evaluate(X_test_pad4, y_test)
print('Test MAE:', scores[0])
y_pred = model1.predict(X_test_pad4).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2 * 100, 2))

90/90 [==============================] - 4s 32ms/step - loss: 27076.8516 - mae: 27076.8516
Test MAE: 27076.8515625
90/90 [==============================] - 3s 26ms/step
R2 score: 63.79


In [ ]:
EMBEDDING_DIM = 256

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)

model=Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(Bidirectional(GRU(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


epochs=15
batch_size=64

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train_pad3, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
180/180 [==============================] - 33s 146ms/step - loss: 46885.0156 - mae: 46885.0156
Epoch 2/15
180/180 [==============================] - 20s 109ms/step - loss: 31657.2246 - mae: 31657.2246
Epoch 3/15
180/180 [==============================] - 18s 98ms/step - loss: 27727.1094 - mae: 27727.1094
Epoch 4/15
180/180 [==============================] - 18s 97ms/step - loss: 25924.4453 - mae: 25924.4453
Epoch 5/15
180/180 [==============================] - 18s 102ms/step - loss: 24750.2480 - mae: 24750.2480
Epoch 6/15
180/180 [==============================] - 18s 98ms/step - loss: 24096.6895 - mae: 24096.6895
Epoch 7/15
180/180 [==============================] - 17s 96ms/step - loss: 23149.0586 - mae: 23149.0586
Epoch 8/15
180/180 [==============================] - 17s 96ms/step - loss: 22426.1328 - mae: 22426.1328
Epoch 9/15
180/180 [==============================] - 17s 96ms/step - loss: 22161.7188 - mae: 22161.7188
Epoch 10/15
180/180 [==============================]

In [ ]:
scores = model.evaluate(X_test_pad3, y_test)
print('Test MAE:', scores[0])
y_pred = model.predict(X_test_pad3).astype(int)
r2 = r2_score(y_test, y_pred)
print('R2 score:', round(r2, 2))

90/90 [==============================] - 3s 26ms/step - loss: 27104.0195 - mae: 27104.0195
Test MAE: 27104.01953125
90/90 [==============================] - 3s 27ms/step
R2 score: 0.62
